## Import packages

In [1]:
from pyspark.sql import DataFrame
from pyspark.ml import *

## Load dataset as DataFrame from JSON

In [2]:
# A JSON dataset is pointed to by path.
# The path can be either a single text file or a directory storing text files
path = "data/nepal_pos.json"
tweetsDF = spark.read.json(path)

In [3]:
# The inferred schema can be visualized using the printSchema() method
tweetsDF.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true

In [8]:
# Creates a temporary view using the DataFrame
tweetsDF.createOrReplaceTempView("tweets")

In [11]:
# SQL statements can be run by using the sql methods provided by spark
tweetTextsDF = spark.sql("SELECT id,text FROM tweets")
tweetTextsDF.count()
#tweetTextsDF.show()

AnalysisException: u"cannot resolve '`label`' given input columns: [favorite_count, contributors, in_reply_to_status_id, in_reply_to_user_id_str, possibly_sensitive, in_reply_to_status_id_str, quoted_status_id, geo, truncated, place, is_quote_status, entities, user, extended_entities, quoted_status_id_str, text, retweet_count, favorited, lang, in_reply_to_screen_name, id, coordinates, in_reply_to_user_id, source, quoted_status, id_str, retweeted, created_at, retweeted_status]; line 1 pos 15;\n'Project [id#8L, text#26, 'label]\n+- SubqueryAlias tweets\n   +- Relation[contributors#0,coordinates#1,created_at#2,entities#3,extended_entities#4,favorite_count#5L,favorited#6,geo#7,id#8L,id_str#9,in_reply_to_screen_name#10,in_reply_to_status_id#11L,in_reply_to_status_id_str#12,in_reply_to_user_id#13L,in_reply_to_user_id_str#14,is_quote_status#15,lang#16,place#17,possibly_sensitive#18,quoted_status#19,quoted_status_id#20L,quoted_status_id_str#21,retweet_count#22L,retweeted#23,... 5 more fields] 

In [18]:
df = spark.read.option("header","true").csv("data/nepal_train.csv")

In [19]:
df = df.select(df.tweet_text,df.label.cast("double").alias("label"))

In [20]:
df.show()

+--------------------+-----+
|          tweet_text|label|
+--------------------+-----+
|in pictures man i...|  0.0|
|still visiting ne...|  1.0|
|sending love to n...|  1.0|
|devastating love ...|  0.0|
|for so many years...|  1.0|
|god this is nepal...|  0.0|
|prayers for #nepa...|  1.0|
|more than killed ...|  0.0|
|like said these c...|  0.0|
|earthquake of mag...|  0.0|
|please tired of r...|  1.0|
|visit this link t...|  0.0|
|      nepal fighting|  0.0|
|may god give stre...|  1.0|
|#israelinnepal nu...|  1.0|
|nepal#kathmanduqu...|  1.0|
|#nepal #uae nnepa...|  0.0|
|breaking at least...|  0.0|
|check out this in...|  1.0|
|indeed our heart ...|  1.0|
+--------------------+-----+
only showing top 20 rows



## Read from text file

In [1]:
from pyspark.sql import Row

In [2]:
# Load a text file and convert each line to a Row.
lines = sc.textFile("data/nepal.csv")
header = lines.first() #extract header
lines = lines.filter(lambda row : row != header)   #filter out header
parts = lines.map(lambda l: l.split(","))
tweets = parts.map(lambda p: Row(tweet_id=p[0], tweet_text=p[1], label=p[2]))

# Infer the schema, and register the DataFrame as a table.
schemaTweets = spark.createDataFrame(tweets)
schemaTweets.createOrReplaceTempView("tweets")
# SQL can be run over DataFrames that have been registered as a table.
all_tweets = spark.sql("SELECT tweet_text, label FROM tweets")
tweetTexts = all_tweets.rdd.map(lambda p: "Text: " + p.tweet_text).collect()
for tweet_text in tweetTexts:
    print(tweet_text)

Py4JJavaError: An error occurred while calling o23.applySchemaToPythonRDD.
: java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
	at org.apache.hadoop.hive.ql.session.SessionState.start(SessionState.java:522)
	at org.apache.spark.sql.hive.client.HiveClientImpl.<init>(HiveClientImpl.scala:189)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.spark.sql.hive.client.IsolatedClientLoader.createClient(IsolatedClientLoader.scala:258)
	at org.apache.spark.sql.hive.HiveUtils$.newClientForMetadata(HiveUtils.scala:359)
	at org.apache.spark.sql.hive.HiveUtils$.newClientForMetadata(HiveUtils.scala:263)
	at o

## Tokenise

In [3]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

tokenizer = Tokenizer(inputCol="tweet_text", outputCol="words")
regexTokenizer = RegexTokenizer(inputCol="tweet_text", outputCol="words", pattern="\\W")
# alternatively, pattern="\\w+", gaps(False)

countTokens = udf(lambda words: len(words), IntegerType())

tokenized = tokenizer.transform(all_tweets)
tokenized.select("tweet_text", "words")\
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)
    
regexTokenized = regexTokenizer.transform(all_tweets)
regexTokenized.select("tweet_text", "words") \
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)    

+----------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------+------+
|tweet_text                                                                                                            |words                                                                                                                                     |tokens|
+----------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------+------+
|"in pictures man is lucky to be rescued as massive earthquake hits nepal"                                             |["in, pictures, man, is, lucky, to, be, rescued, as, massive, earthquake, hits,

## Stop words remover

In [4]:
from pyspark.ml.feature import StopWordsRemover

remover = StopWordsRemover(inputCol="words", outputCol="filtered")
remover.transform(tokenized).show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------+-----+------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------+
|tweet_text                                                                                                            |label|words                                                                                                                                     |filtered                                                                                                                 |
+----------------------------------------------------------------------------------------------------------------------+-----+--------------------------------------------------------------------------------------------------

## NGram

In [5]:
from pyspark.ml.feature import NGram

ngram = NGram(n=3, inputCol="words", outputCol="ngrams")

ngramDataFrame = ngram.transform(tokenized)
ngramDataFrame.select("ngrams").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ngrams                                                                                                                                                                                                                                                                                                                            |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|["in pictures man, pictu

## HashingTF and IDF

In [6]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.mllib.regression import LabeledPoint

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=50000)
featurizedData = hashingTF.transform(tokenized)
# alternatively, CountVectorizer can also be used to get term frequency vectors
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.select("label", "tweet_text", "features").show()

+-----+--------------------+--------------------+
|label|          tweet_text|            features|
+-----+--------------------+--------------------+
|    0|"in pictures man ...|(50000,[518,2836,...|
|    1|"still visiting n...|(50000,[3888,8352...|
|    1|"sending love to ...|(50000,[6240,8388...|
|    0|"devastating love...|(50000,[6240,8388...|
|    1|"for so many year...|(50000,[3138,4725...|
|    0|"god this is nepa...|(50000,[2836,3888...|
|    1|"prayers for #nep...|(50000,[4725,6683...|
|    0|"more than killed...|(50000,[590,3440,...|
|    0|"like said these ...|(50000,[3542,5833...|
|    0|"earthquake of ma...|(50000,[3888,1266...|
|    1|"please tired of ...|(50000,[3514,4725...|
|    0|"visit this link ...|(50000,[1198,5833...|
|    0|    "nepal fighting"|(50000,[13578,365...|
|    1|"may god give str...|(50000,[2836,8388...|
|    1|"#israelinnepal n...|(50000,[1583,8388...|
|    1|"nepal#kathmanduq...|(50000,[8388,1697...|
|    0|"#nepal #uae \nne...|(50000,[2762,4959...|


In [9]:
from pyspark.mllib.regression import LabeledPoint
labeled_RDD = lines.zip(rescaledData).map(lambda couple: LabeledPoint(couple[0],couple[1]))

AttributeError: 'DataFrame' object has no attribute '_jrdd_deserializer'

In [ ]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression



In [ ]:
from pyspark.mllib.util import MLUtils

#MLUtils.saveAsLibSVMFile(dataRDD, dir)

## From Text to LIBsvm

In [1]:
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes   

training_raw = sc.parallelize([
    {"text": "foo foo foo bar bar protein", "label": 1.0},
    {"text": "foo bar dna for bar", "label": 0.0},
    {"text": "foo bar foo dna foo", "label": 0.0},
    {"text": "bar foo protein foo ", "label": 1.0}])

labels = training_raw.map(
    lambda doc: doc["label"],  # Standard Python dict access 
    preservesPartitioning=True # This is obsolete.
)

tf = HashingTF(numFeatures=100).transform( ## Use much larger number in practice
    training_raw.map(lambda doc: doc["text"].split(), 
    preservesPartitioning=True))

idf = IDF().fit(tf)
tfidf = idf.transform(tf)

# Combine using zip
training = labels.zip(tfidf).map(lambda x: LabeledPoint(x[0], x[1]))

# Train and check
model = NaiveBayes.train(training)

In [2]:
tf

PythonRDD[16] at RDD at PythonRDD.scala:48

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

from pyspark.ml import Pipeline

pipeline = Pipeline(stages = [tokenizer, hashingTF, lr])

model = pipeline.fit(all_tweets)